In [ ]:
import tensorflow as tf
import open3d as o3d
import numpy as np
from models.pcn import *
from models.pccn import *
import os
import open3d_util

print(tf.__version__)
print(o3d.__version__)

## Dataloader Development

In [ ]:
from datagenerator import DataGenerator

def get_file_paths(complete_dir, training=True):
    mid_pt = int(len(os.listdir(complete_dir))*7/9)
    start = 0 if training else mid_pt
    end = mid_pt if training else None
    return sorted(os.listdir(complete_dir))[start:end]

complete_dir = "data/complete/"
ds_train = DataGenerator(get_file_paths(complete_dir), complete_dir)
print(len(ds_train))

In [9]:
for idx, (inputs, gt) in enumerate(ds_train):
    print(sum(inputs[1]) == inputs[0].shape[1], end=" ")

True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True 

In [10]:
def foo():
    return (1, 2), 3
a, b = foo()
print(a, b)
q,w = a
print(q,w)

(1, 2) 3
1 2


In [ ]:
print(len(ds_valid.list_pcd_path)//8)
c = 0
for id, data in enumerate(ds_valid):
    assert sum(data[1]) == data[0].shape[1]
    c+=1
    if c % 10: print("-",end=" ")
    # print(data[0].shape, end=" | ")
print(c, ds_iter.counter)

In [ ]:
import tensorflow as tf
from dataloader import Dataloader
ds_train = Dataloader(complete_dir="data/complete/", is_training=True, batch_size=1)
ds_train = tf.data.Dataset.from_generator(ds_train, output_types=(tf.float32, tf.int32, tf.float32), output_shapes=((), (), ()))

In [ ]:
from dataloader_old import Dataloader
ds_train = Dataloader("data/complete/", is_training=False)
a, b, c = ds_train.split_to_batch()

In [ ]:
import tensorflow as tf
q, w, e = tf.convert_to_tensor(a), tf.convert_to_tensor(b), tf.convert_to_tensor(c)

In [ ]:
from models.pcn import PCN
model = PCN()
model.build(input_shape=(1,None,3))
model.summary()

## Utils Function

In [ ]:
def pprint(s, shape):
    print(s.center(35), ":",shape)

## PCN Development

In [ ]:
def get_file_paths(complete_dir, training=True):
    mid_pt = int(len(os.listdir(complete_dir))*7/9)
    start = 0 if training else mid_pt
    end = mid_pt if training else None
    return sorted(os.listdir(complete_dir))[start:end]

len(get_file_paths("data/complete/", training=0))

def xform(dir):
    return dir.replace("complete", "partial")

def get_pcds_np(complete_dir, training=True):
    partial, npts, complete = [], [], []
    cnt = 0
    for pcd_path in get_file_paths(complete_dir, training=True):
        complete.append(open3d_util.read_pcd(os.path.join(complete_dir, pcd_path)))
        partial.append(open3d_util.read_pcd(os.path.join(xform(complete_dir), xform(pcd_path))))
        npts.append(len(partial[-1]))
        cnt+=1
        if cnt%100 == 0: print("-",end=" ")
    return partial, npts, complete

partial, npts, complete = get_pcds_np("data/complete/", training=True)

import numpy as np
import os
checkpoint_name = "cp-{step}.ckpt"
checkpoint_name.format(step=10)
os.path.join("training",checkpoint_name.format(step=100))

a = [i for i in range(10)]
np.random.shuffle(a)
print(a)

In [ ]:
idx = np.random.permutation(24)
print(idx)
print(idx[:None])

def get_idx(idxs, bs):
    ids = []
    for i in range(int(np.ceil(len(idxs)/bs))):
        start = i*bs
        end = None if start+bs >= len(idxs) else start+bs
        print(start, end)
        ids.append(idxs.tolist()[start:end])
    return ids
print(get_idx(idx, 8))


In [ ]:
# ids, inputs, npts, gt
npts = [20, 30, 55]
partial = tf.random.normal((1,105,3))
complete = tf.random.normal((3,16384,3))

model = PCN()
coarse, fine = model((partial,npts), training=True)
pprint("Output Tensor (aft decoder): coarse-->", coarse.shape)
pprint("Output Tensor (aft decoder): fine-->", fine.shape)

In [ ]:
from loss_utils.emd import *

a = tf.random.normal((3,123,3))
b = tf.random.normal((3,123,3))

print( getEMD(a.numpy(),b.numpy()) )

In [ ]:
def mlp_conv(inputs, channels):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv1D(channels[0], 1, activation='relu'))
    model.add(tf.keras.layers.Conv1D(channels[1], 1, activation='relu'))
    outputs = model(inputs)
    return outputs

def mlp(inputs, channels):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(channels[0], activation='relu'))
    model.add(tf.keras.layers.Dense(channels[1], activation='relu'))
    model.add(tf.keras.layers.Dense(channels[2], activation='relu'))
    outputs = model(inputs)
    return outputs

def point_maxpool(inputs, npts, keepdims=False):
    outputs = [tf.reduce_max(f, axis=1, keepdims=keepdims)
        for f in tf.split(inputs, npts, axis=1)]
    return tf.concat(outputs, axis=0)


def point_unpool(inputs, npts):
    inputs = tf.split(inputs, inputs.shape[0], axis=0)
    outputs = [tf.tile(f, [1, npts[i], 1]) for i,f in enumerate(inputs)]
    return tf.concat(outputs, axis=1)

# npts = [1024,1024,1024]
# x = tf.random.normal((3,1024,3))
# pprint("Input Tensor", x.shape)
# x = tf.reshape(x, [1, x.shape[1]*x.shape[0], 3])
npts = [20, 30, 55]
x = tf.random.normal((1,105,3))
pprint("Input Tensor (aft reshaping)", x.shape)
features = mlp_conv(x,[64,128])
pprint("Output Tensor (aft Conv1D)", features.shape)

features_1 = point_maxpool(features, npts, keepdims=1)
pprint("Output Tensor (aft maxpool)", features_1.shape)
features_global = point_unpool(features_1, npts)
pprint("Output Tensor (aft unpool)", features_global.shape)

features = tf.concat([features, features_global], axis=2)
pprint("Output Tensor (aft concat)", features.shape)

features = mlp_conv(features,[512,1024])
pprint("Output Tensor (aft 2nd conv)", features.shape)
features = point_maxpool(features,npts)
pprint("Output Tensor (aft 2nd maxpool)", features.shape)


coarse = mlp(features,[1024,1024, 1024*3])
pprint("Output Tensor (aft MLP)", coarse.shape)

coarse = tf.reshape(coarse, [-1, 1024, 3])
pprint("Output Tensor (aft reshaping)", coarse.shape)

"""
    Input Tensor (aft reshaping)    : (1, 105, 3)
     Output Tensor (aft Conv1D)     : (1, 105, 128)
    Output Tensor (aft maxpool)     : (3, 1, 128)
     Output Tensor (aft unpool)     : (1, 105, 128)
     Output Tensor (aft concat)     : (1, 105, 256)
    Output Tensor (aft 2nd conv)    : (1, 105, 1024)
  Output Tensor (aft 2nd maxpool)   : (3, 1024)
      Output Tensor (aft MLP)       : (3, 3072)
   Output Tensor (aft reshaping)    : (3, 1024, 3)
"""


## PCCN Development

Input Tensor (aft reshaping)      : (1, 105, 3)
Output Tensor (aft Conv1D)        : (1, 105, 128)
Output Tensor (aft maxpool)       : (3, 1, 128)
Output Tensor (aft unpool)        : (1, 105, 128)
Output Tensor (aft concat)        : (1, 105, 256)
Output Tensor (aft 2nd conv)      : (1, 105, 1024)
Output Tensor (aft 2nd maxpool)   : (3, 1024)

In [ ]:
partial = tf.random.normal((1,12,3))
x = tf.expand_dims(partial, axis=0)
print(x)

In [ ]:
x = tf.keras.layers.Conv2D(5, [1,3], activation='relu')(x)
print(x)

In [ ]:
import tensorflow as tf

npts = [20, 30, 55]
partial = tf.random.normal((1,105,3))
complete = tf.random.normal((3,16384,3))

x = tf.expand_dims(partial, axis=-1)
pprint("Input Tensor (aft expansion)", x.shape)
x = tf.keras.layers.Conv2D(64, [1,3], padding='valid', activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(128, 1,activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(256, 1, activation='relu')(x)
print(x.shape)
x = tf.keras.layers.MaxPool2D(pool_size=[105, 1], strides=(2, 2), padding='valid')(x)
print(x.shape)

In [ ]:
input_shape = (12, 189, 3, 1)
bs = input_shape[0]
num_pts = input_shape[1]

# Point functions (MLP implemented as conv2d)
# By default, Keras version of Conv2D has: strides=[1,1], padding="valid" , data_format="channels_last"
x = tf.random.normal(input_shape)
x = tf.keras.layers.Conv2D(64, [1,3], activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(64, [1,1],activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(64, [1,1], activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(128, [1,1], activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(1024, [1,1], activation='relu')(x)
print(x.shape)

# Symmetric function: max pooling
x = tf.keras.layers.MaxPool2D(pool_size=[num_pts, 1], strides=(2, 2), padding='valid')(x)
print(x.shape)

# MLP on global point cloud vector
x = tf.reshape(x, [bs, -1])
print(x.shape)
x = tf.keras.layers.Dense(1024,activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Dense(512,activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Dense(256,activation='relu')(x)
print(x.shape)

mean, logvar = tf.split(x, num_or_size_splits=2, axis=1)
print("Mean:",mean.shape, " Log(Var):",logvar.shape)